##### Copyright 2021 The TensorFlow Hub Authors. 
Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# 식물 질병 감지를 위한 미세 조정 모델


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cropnet_on_device"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/cropnet_on_device.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/cropnet_on_device.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View on GitHub</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/cropnet_on_device.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
  <td>     <a href="https://tfhub.dev/s?module-type=image-feature-vector&amp;q=cropnet"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">See TF Hub models</a>   </td>
</table>

이 노트북은 TFDS의 데이터 세트 또는 농작물 질병 감지 데이터세트에서 **TensorFlow Hub의 CropNet 모델을 미세 조정**하는 방법을 보여줍니다.

이를 통해 다음을 수행할 수 있습니다.

- TFDS 카사바 데이터세트 또는 자체 데이터 로드
- 더 강력한 모델을 얻기 위해 알려지지 않은(음성) 예제로 데이터 보강
- 데이터에 이미지 강화 적용
- Load and fine tune a [CropNet model](https://tfhub.dev/s?module-type=image-feature-vector&q=cropnet) from TF Hub
- [작업 라이브러리](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier), [MLKit](https://developers.google.com/ml-kit/vision/image-labeling/custom-models/android) 또는 [TFLite](https://www.tensorflow.org/lite/guide/inference)를 사용하여 앱에 직접 배포할 준비가 된 TFLite 모델 내보내기

## 가져오기 및 종속성

Before starting, you'll need to install some of the dependencies that will be needed like [Model Maker](https://www.tensorflow.org/lite/guide/model_maker#installation) and the latest version of TensorFlow Datasets.

In [ ]:
!sudo apt install -q libportaudio2
!pip install --use-deprecated=legacy-resolver tflite-model-maker-nightly
!pip install -U tensorflow-datasets
## scann library requires tensorflow < 2.9.0
!pip install "tensorflow<2.9.0"

In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow_examples.lite.model_maker.core.export_format import ExportFormat
from tensorflow_examples.lite.model_maker.core.task import image_preprocessing

from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker.image_classifier import ModelSpec

## 미세 조정할 TFDS 데이터세트 로드

TFDS에서 공개적으로 사용 가능한 [카사바 잎 질병 데이터세트](https://www.tensorflow.org/datasets/catalog/cassava)를 사용하도록 하겠습니다.

In [ ]:
tfds_name = 'cassava'
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    name=tfds_name,
    split=['train', 'validation', 'test'],
    with_info=True,
    as_supervised=True)
TFLITE_NAME_PREFIX = tfds_name

## 또는 미세 조정할 자체 데이터 대체 로드

TFDS 데이터세트를 사용하는 대신 자체 데이터로 훈련할 수도 있습니다. 이 코드 스니펫은 사용자 정의 데이터세트를 로드하는 방법을 보여줍니다. 지원되는 데이터 구조는 [이](https://www.tensorflow.org/datasets/api_docs/python/tfds/folder_dataset/ImageFolder) 링크를 참조하십시오. 다음에는 공개적으로 사용 가능한 [카사바 잎 질병 데이터세트](https://www.tensorflow.org/datasets/catalog/cassava)를 사용한 예가 제공되어 있습니다.

In [ ]:
# data_root_dir = tf.keras.utils.get_file(
#     'cassavaleafdata.zip',
#     'https://storage.googleapis.com/emcassavadata/cassavaleafdata.zip',
#     extract=True)
# data_root_dir = os.path.splitext(data_root_dir)[0]  # Remove the .zip extension

# builder = tfds.ImageFolder(data_root_dir)

# ds_info = builder.info
# ds_train = builder.as_dataset(split='train', as_supervised=True)
# ds_validation = builder.as_dataset(split='validation', as_supervised=True)
# ds_test = builder.as_dataset(split='test', as_supervised=True)

## 훈련 분할에서 샘플 시각화

이미지 샘플 및 해당 레이블에 대한 클래스 ID와 클래스 이름을 포함하는 데이터세트의 몇 가지 예를 살펴보겠습니다.

In [ ]:
_ = tfds.show_examples(ds_train, ds_info)

## TFDS 데이터세트에서 알 수 없는 예시로 사용할 이미지 추가

학습 데이터세트에 알 수 없는(음성) 예제를 추가하고 알 수 없는 새 클래스 레이블 번호를 할당합니다. 예를 들어 실제로 현장에서 사용될 때 예상치 못한 것을 볼 때 "알 수 없음"을 예측하는 옵션이 있는 모델을 보유하기 위함입니다.

아래에서 알 수 없는 추가 이미지를 샘플링하는 데 사용할 데이터세트 목록을 볼 수 있습니다. 다양성을 높이기 위해 전혀 다른 3개의 데이터세트가 포함되어 있습니다. 그 중 하나는 콩 잎 질병 데이터세트이므로 해당 모델은 카사바 이외의 병든 식물에 노출됩니다.


In [ ]:
UNKNOWN_TFDS_DATASETS = [{
    'tfds_name': 'imagenet_v2/matched-frequency',
    'train_split': 'test[:80%]',
    'test_split': 'test[80%:]',
    'num_examples_ratio_to_normal': 1.0,
}, {
    'tfds_name': 'oxford_flowers102',
    'train_split': 'train',
    'test_split': 'test',
    'num_examples_ratio_to_normal': 1.0,
}, {
    'tfds_name': 'beans',
    'train_split': 'train',
    'test_split': 'test',
    'num_examples_ratio_to_normal': 1.0,
}]

UNKNOWN 데이터세트도 TFDS에서 로드됩니다.

In [ ]:
# Load unknown datasets.
weights = [
    spec['num_examples_ratio_to_normal'] for spec in UNKNOWN_TFDS_DATASETS
]
num_unknown_train_examples = sum(
    int(w * ds_train.cardinality().numpy()) for w in weights)
ds_unknown_train = tf.data.Dataset.sample_from_datasets([
    tfds.load(
        name=spec['tfds_name'], split=spec['train_split'],
        as_supervised=True).repeat(-1) for spec in UNKNOWN_TFDS_DATASETS
], weights).take(num_unknown_train_examples)
ds_unknown_train = ds_unknown_train.apply(
    tf.data.experimental.assert_cardinality(num_unknown_train_examples))
ds_unknown_tests = [
    tfds.load(
        name=spec['tfds_name'], split=spec['test_split'], as_supervised=True)
    for spec in UNKNOWN_TFDS_DATASETS
]
ds_unknown_test = ds_unknown_tests[0]
for ds in ds_unknown_tests[1:]:
  ds_unknown_test = ds_unknown_test.concatenate(ds)

# All examples from the unknown datasets will get a new class label number.
num_normal_classes = len(ds_info.features['label'].names)
unknown_label_value = tf.convert_to_tensor(num_normal_classes, tf.int64)
ds_unknown_train = ds_unknown_train.map(lambda image, _:
                                        (image, unknown_label_value))
ds_unknown_test = ds_unknown_test.map(lambda image, _:
                                      (image, unknown_label_value))

# Merge the normal train dataset with the unknown train dataset.
weights = [
    ds_train.cardinality().numpy(),
    ds_unknown_train.cardinality().numpy()
]
ds_train_with_unknown = tf.data.Dataset.sample_from_datasets(
    [ds_train, ds_unknown_train], [float(w) for w in weights])
ds_train_with_unknown = ds_train_with_unknown.apply(
    tf.data.experimental.assert_cardinality(sum(weights)))

print((f"Added {ds_unknown_train.cardinality().numpy()} negative examples."
       f"Training dataset has now {ds_train_with_unknown.cardinality().numpy()}"
       ' examples in total.'))

## 강화 적용

모든 이미지를 다양하게 만들기 위해 다음 변경 사항 같은 몇 가지 강화 기능을 적용합니다.

- 밝기
- 대비
- 포화
- 색조
- 절단

이러한 유형의 강화 기능은 모델을 이미지 입력의 변화에 대해 더 강력하게 만드는 데 도움이 됩니다.


In [ ]:
def random_crop_and_random_augmentations_fn(image):
  # preprocess_for_train does random crop and resize internally.
  image = image_preprocessing.preprocess_for_train(image)
  image = tf.image.random_brightness(image, 0.2)
  image = tf.image.random_contrast(image, 0.5, 2.0)
  image = tf.image.random_saturation(image, 0.75, 1.25)
  image = tf.image.random_hue(image, 0.1)
  return image


def random_crop_fn(image):
  # preprocess_for_train does random crop and resize internally.
  image = image_preprocessing.preprocess_for_train(image)
  return image


def resize_and_center_crop_fn(image):
  image = tf.image.resize(image, (256, 256))
  image = image[16:240, 16:240]
  return image


no_augment_fn = lambda image: image

train_augment_fn = lambda image, label: (
    random_crop_and_random_augmentations_fn(image), label)
eval_augment_fn = lambda image, label: (resize_and_center_crop_fn(image), label)

강화 기능을 적용하기 위해 Dataset 클래스의 `map` 메소드를 사용합니다.

In [ ]:
ds_train_with_unknown = ds_train_with_unknown.map(train_augment_fn)
ds_validation = ds_validation.map(eval_augment_fn)
ds_test = ds_test.map(eval_augment_fn)
ds_unknown_test = ds_unknown_test.map(eval_augment_fn)

## 데이터를 Model Maker 친화적인 형식으로 래핑

Model Maker와 함께 이러한 데이터세트를 사용하려면 데이터세트가 ImageClassifierDataLoader 클래스에 있어야 합니다.

In [ ]:
label_names = ds_info.features['label'].names + ['UNKNOWN']

train_data = ImageClassifierDataLoader(ds_train_with_unknown,
                                       ds_train_with_unknown.cardinality(),
                                       label_names)
validation_data = ImageClassifierDataLoader(ds_validation,
                                            ds_validation.cardinality(),
                                            label_names)
test_data = ImageClassifierDataLoader(ds_test, ds_test.cardinality(),
                                      label_names)
unknown_test_data = ImageClassifierDataLoader(ds_unknown_test,
                                              ds_unknown_test.cardinality(),
                                              label_names)

## 훈련 실행하기

[TensorFlow Hub](https://tfhub.dev)에는 전이 학습에 사용할 수 있는 여러 가지 모델이 있습니다.

여기에서 하나를 선택할 수 있으며 더 나은 결과를 얻기 위해 다른 것들을 계속 실험할 수도 있습니다.

더 많은 모델을 시도하고 싶다면 이 [컬렉션](https://tfhub.dev/google/collections/image/1)에서 추가할 수 있습니다.


In [ ]:
#@title Choose a base model

model_name = 'mobilenet_v3_large_100_224'  #@param ['cropnet_cassava', 'cropnet_concat', 'cropnet_imagenet', 'mobilenet_v3_large_100_224']

map_model_name = {
    'cropnet_cassava':
        'https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1',
    'cropnet_concat':
        'https://tfhub.dev/google/cropnet/feature_vector/concat/1',
    'cropnet_imagenet':
        'https://tfhub.dev/google/cropnet/feature_vector/imagenet/1',
    'mobilenet_v3_large_100_224':
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
}

model_handle = map_model_name[model_name]

모델을 미세 조정하기 위해 Model Maker를 사용합니다. 이렇게 하면 모델을 훈련한 후 TFLite로 변환하기 때문에 전체 솔루션이 더 쉬워집니다.

Model Maker는 나중에 기기에 모델을 쉽게 배포하는 데 필요한 모든 정보를 사용하여 이러한 변환을 최상의 상태로 만듭니다.

모델 사양은 Model Maker에 사용하려는 기본 모델을 알려주는 방법입니다.

In [ ]:
image_model_spec = ModelSpec(uri=model_handle)

여기서 중요한 세부 사항 중 하나는 훈련 중에 기본 모델을 미세 조정하도록 `train_whole_model`을 설정하는 것입니다. 이로 인해 프로세스가 느려지지만 최종 모델의 정확도는 더 높아집니다. `shuffle`을 설정하면 무작위로 섞인 순서로 데이터를 볼 수 있습니다. 이는 모델 학습을 위한 모범 사례입니다.

In [ ]:
model = image_classifier.create(
    train_data,
    model_spec=image_model_spec,
    batch_size=128,
    learning_rate=0.03,
    epochs=5,
    shuffle=True,
    train_whole_model=True,
    validation_data=validation_data)

## 테스트 분할에서 모델 평가하기

In [ ]:
model.evaluate(test_data)

미세 조정된 모델을 더 잘 이해하려면 혼동 행렬을 분석하는 것이 좋습니다. 이것은 한 클래스가 다른 클래스로 예측되는 빈도를 보여줍니다.

In [ ]:
def predict_class_label_number(dataset):
  """Runs inference and returns predictions as class label numbers."""
  rev_label_names = {l: i for i, l in enumerate(label_names)}
  return [
      rev_label_names[o[0][0]]
      for o in model.predict_top_k(dataset, batch_size=128)
  ]

def show_confusion_matrix(cm, labels):
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, xticklabels=labels, yticklabels=labels, 
              annot=True, fmt='g')
  plt.xlabel('Prediction')
  plt.ylabel('Label')
  plt.show()

In [ ]:
confusion_mtx = tf.math.confusion_matrix(
    list(ds_test.map(lambda x, y: y)),
    predict_class_label_number(test_data),
    num_classes=len(label_names))

show_confusion_matrix(confusion_mtx, label_names)

## 알 수 없는 테스트 데이터에서 모델 평가하기

이 평가에서 우리는 모델이 거의 1의 정확도를 가질 것으로 예상합니다. 모델이 테스트된 모든 이미지는 일반 데이터세트와 관련이 없으므로 모델이 "알 수 없는" 클래스 레이블을 예측할 것으로 예상합니다.

In [ ]:
model.evaluate(unknown_test_data)

혼동 행렬을 인쇄합니다.

In [ ]:
unknown_confusion_mtx = tf.math.confusion_matrix(
    list(ds_unknown_test.map(lambda x, y: y)),
    predict_class_label_number(unknown_test_data),
    num_classes=len(label_names))

show_confusion_matrix(unknown_confusion_mtx, label_names)

## 모델을 TFLite 및 SavedModel로 내보내기

이제 기기에 배포하고 TensorFlow에서 추론에 사용하기 위해 훈련된 모델을 TFLite 및 SavedModel 형식으로 내보낼 수 있습니다.

In [ ]:
tflite_filename = f'{TFLITE_NAME_PREFIX}_model_{model_name}.tflite'
model.export(export_dir='.', tflite_filename=tflite_filename)

In [ ]:
# Export saved model version.
model.export(export_dir='.', export_format=ExportFormat.SAVED_MODEL)

## Next steps

방금 훈련한 모델을 모바일 기기에서 사용하고 현장에 배포할 수도 있습니다!

**모델을 다운로드하려면 colab 왼쪽에 있는 파일 메뉴의 폴더 아이콘을 클릭하고 다운로드 옵션을 선택하십시오.**

여기에 사용된 동일한 기술을 사용 사례 또는 다른 유형의 이미지 분류 작업에 더 적합할 수 있는 다른 식물 질병 작업에 적용할 수 있습니다. Android 앱에서 후속 작업을 수행하고 배포하려는 경우 이 [Android 빠른 시작 가이드](https://www.tensorflow.org/lite/android/quickstart)에서 계속 진행할 수 있습니다.